In [1]:
%matplotlib inline


`Learn the Basics <intro.html>`_ ||
`Quickstart <quickstart_tutorial.html>`_ ||
`Tensors <tensorqs_tutorial.html>`_ ||
`Datasets & DataLoaders <data_tutorial.html>`_ ||
`Transforms <transforms_tutorial.html>`_ ||
**Build Model** ||
`Autograd <autogradqs_tutorial.html>`_ ||
`Optimization <optimization_tutorial.html>`_ ||
`Save & Load Model <saveloadrun_tutorial.html>`_

Build the Neural Network
===================

Neural networks comprise of layers/modules that perform operations on data.
The `torch.nn <https://pytorch.org/docs/stable/nn.html>`_ namespace provides all the building blocks you need to
build your own neural network. Every module in PyTorch subclasses the `nn.Module <https://pytorch.org/docs/stable/generated/torch.nn.Module.html>`_.
A neural network is a module itself that consists of other modules (layers). This nested structure allows for
building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.




In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
-----------------------
We want to be able to train our model on a hardware accelerator like the GPU,
if it is available. Let's check to see if
`torch.cuda <https://pytorch.org/docs/stable/notes/cuda.html>`_ is available, else we
continue to use the CPU.



In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


Define the Class
-------------------------
We define our neural network by subclassing ``nn.Module``, and
initialize the neural network layers in ``__init__``. Every ``nn.Module`` subclass implements
the operations on input data in the ``forward`` method.



In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of ``NeuralNetwork``, and move it to the ``device``, and print
its structure.



In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model's ``forward``,
along with some `background operations <https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866>`_.
Do not call ``model.forward()`` directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class.
We get the prediction probabilities by passing it through an instance of the ``nn.Softmax`` module.



In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0], device='cuda:0')


--------------




Model Layers
-------------------------

Let's break down the layers in the FashionMNIST model. To illustrate it, we
will take a sample minibatch of 3 images of size 28x28 and see what happens to it as
we pass it through the network.



In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
^^^^^^^^^^^^^^^^^^^^^^
We initialize the `nn.Flatten  <https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html>`_
layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (
the minibatch dimension (at dim=0) is maintained).



In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
^^^^^^^^^^^^^^^^^^^^^^
The `linear layer <https://pytorch.org/docs/stable/generated/torch.nn.Linear.html>`_
is a module that applies a linear transformation on the input using its stored weights and biases.




In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
^^^^^^^^^^^^^^^^^^^^^^
Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use `nn.ReLU <https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html>`_ between our
linear layers, but there's other activations to introduce non-linearity in your model.



In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2146, -0.6965, -0.4252,  0.0903, -0.0466, -0.3215, -0.9861, -0.5393,
         -0.2074,  0.5491, -0.5353,  0.0403, -0.1675, -0.4454,  0.1581,  0.0154,
          0.2117,  0.3410,  0.0695, -0.3789],
        [-0.2117, -0.2437, -0.0539,  0.3325, -0.3976, -0.0559, -0.7735, -0.5841,
         -0.1599,  0.6433, -0.4822,  0.3598, -0.2377, -0.5862, -0.0065,  0.4646,
          0.7632,  0.7807, -0.0037, -0.5790],
        [-0.0794, -0.2082, -0.2818, -0.0447, -0.1115, -0.2147, -0.7289, -0.5859,
         -0.2715,  0.5436, -0.6611,  0.2494, -0.1367, -0.6979,  0.2633,  0.3108,
          0.2322,  0.5778, -0.2090, -0.2005]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0903, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.5491, 0.0000, 0.0403, 0.0000, 0.0000, 0.1581, 0.0154, 0.2117, 0.3410,
         0.0695, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.3325, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.6433, 0.0000, 0.3598, 0.0000, 0.0000, 0.000

nn.Sequential
^^^^^^^^^^^^^^^^^^^^^^
`nn.Sequential <https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html>`_ is an ordered
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like ``seq_modules``.



In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax
^^^^^^^^^^^^^^^^^^^^^^
The last linear layer of the neural network returns `logits` - raw values in [-\infty, \infty] - which are passed to the
`nn.Softmax <https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html>`_ module. The logits are scaled to values
[0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along
which the values must sum to 1.



In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
-------------------------
Many layers inside a neural network are *parameterized*, i.e. have associated weights
and biases that are optimized during training. Subclassing ``nn.Module`` automatically
tracks all fields defined inside your model object, and makes all parameters
accessible using your model's ``parameters()`` or ``named_parameters()`` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.




In [13]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0172, -0.0081,  0.0291,  ...,  0.0308,  0.0281,  0.0204],
        [-0.0203,  0.0127, -0.0007,  ..., -0.0311, -0.0163, -0.0304]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0184, -0.0197], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0229,  0.0060, -0.0048,  ..., -0.0229,  0.0333,  0.0056],
        [ 0.0114, -0.0430, -0.0280,  ..., -0.0351, -0.0247,  0.0387]],
       device='cuda:0', grad_fn=<Sl

--------------




In [16]:
for param in model.parameters():
    print(f"Layer:  | Size: {param.size()} | Values : {param[:2]} \n")
    print(param)

Parameter containing:
tensor([[-0.0172, -0.0081,  0.0291,  ...,  0.0308,  0.0281,  0.0204],
        [-0.0203,  0.0127, -0.0007,  ..., -0.0311, -0.0163, -0.0304],
        [ 0.0145,  0.0224,  0.0097,  ..., -0.0293, -0.0086, -0.0043],
        ...,
        [ 0.0186,  0.0092, -0.0347,  ..., -0.0338, -0.0155,  0.0227],
        [ 0.0280, -0.0194,  0.0188,  ..., -0.0253, -0.0155, -0.0190],
        [ 0.0195, -0.0025, -0.0280,  ...,  0.0006,  0.0211, -0.0183]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 1.8419e-02, -1.9688e-02,  3.9215e-03,  8.2161e-03, -1.7048e-02,
        -3.8919e-03, -7.7686e-03, -9.3847e-03,  2.8087e-03, -1.8687e-02,
         3.4807e-02, -2.9393e-02,  8.9519e-03,  3.4492e-02,  3.1996e-03,
         1.3750e-02,  6.7979e-03,  8.5280e-03, -2.6479e-02, -2.0055e-02,
         1.3439e-02, -7.5949e-05, -3.9552e-03, -2.7133e-02, -2.1163e-03,
         3.2017e-02,  1.7252e-02,  4.7899e-03, -2.4084e-02,  2.5570e-02,
         4.9577e-03,  6.4330e-03, -9.746

Further Reading
--------------
- `torch.nn API <https://pytorch.org/docs/stable/nn.html>`_

